In [1]:
include("ABLBreIF/BLBreIF.jl")
include("SCNMF/SCNMF.jl")
using .BLBreIF
using .SCNMF

using SparseArrays

In [2]:
A = sprand(Float64, 200, 200, 0.8)
A = SCNMF.normalize!(A)
X, Y = SCNMF.randinit(A, 10, 0.2)

([0.02335970200127703 0.020124957244744036 … 0.0027832256540637063 0.029966070722195474; 0.0 0.01784733703771187 … 0.0 0.004606051982127539; … ; 0.033833988951441726 0.03850590151334977 … 0.0008666633032087298 0.016121986048718125; 0.0037542149350897856 0.0029526204781013987 … 0.011628442831765612 0.029385908431876467], [0.02538411275339426 0.030356557579757224 … 0.0 0.0; 0.034450230634050207 0.00401924501292998 … 0.020217279950704083 0.0052151114808649994; … ; 0.019175317442394646 0.01743663874478375 … 0.022961173400374832 0.024435725901350725; 0.03244935570630669 0.0 … 0.007859253171110652 0.009634044598935668])

In [3]:
ρ₀ = 0.8 # step size parameter
μ₀ = 0.05 # regularization cofficient
rtime = 15 # runtime

15

In [4]:
X₀ = copy(X);
Y₀ = copy(Y);
r₀ = SCNMF.solve!(SCNMF.BPG{Float64}(obj=:cons,
        constraint=0.8,
        runtime=rtime,
        ρ=ρ₀), A, X₀, Y₀)

X₁ = copy(X);
Y₁ = copy(Y);
r₁ = SCNMF.solve!(SCNMF.BBPG{Float64}(obj=:cons,
        constraint=0.8,
        runtime=rtime,
        ρ=ρ₀), A, X₁, Y₁)

X₂ = copy(X);
Y₂ = copy(Y);
r₂ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
        verbose=true,
        ρ=ρ₀,
        μ₁=μ₀,
        μ₂=μ₀), A, X₂, Y₂)

X₃ = copy(X);
Y₃ = copy(Y);
r₃ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
        verbose=true,
        ρ=ρ₀,
        μ₁=μ₀,
        μ₂=μ₀), A, X₃, Y₃)

MethodError: MethodError: no method matching Main.SCNMF.BPG{Float64}(; obj=:cons, verbose=true, constraint=0.8, runtime=15, ρ=0.8)
Closest candidates are:
  Main.SCNMF.BPG{T}(; obj, constraint, runtime, ρ) where T at d:\gitee\BLBreIF\SCNMF\BPG.jl:7 got unsupported keyword argument "verbose"

In [5]:
stop₀ = r₀.niters
pic₀ = r₀.objvalue
rt₀ = pic₀[1:stop₀, 1]
obj₀ = pic₀[1:stop₀, 2]

stop₁ = r₁.niters
pic₁ = r₁.objvalue
rt₁ = pic₁[1:stop₁, 1]
obj₁ = pic₁[1:stop₁, 2]

stop₂ = r₂.niters
pic₂ = r₂.objvalue
rt₂ = pic₂[1:stop₂, 1]
obj₂ = pic₂[1:stop₂, 2]

stop₃ = r₃.niters
pic₃ = r₃.objvalue
rt₃ = pic₃[1:stop₃, 1]
obj₃ = pic₃[1:stop₃, 2]

UndefVarError: UndefVarError: r₀ not defined

In [6]:
using CairoMakie
using Makie
using LaTeXStrings
using GLMakie
using Colors
using AlgebraOfGraphics
CairoMakie.activate!()
function speed()
    lines(rt₀, sqrt.(2*obj₀); color="#4063D8", label="BPG", linewidth=3, linestyle=:dashdot,
            figure=(; figure_padding=50, resolution=(1200, 800), font="sans",
                    backgroundcolor=:white, fontsize=32),
            axis=(; xlabel="Time(sec)", ylabel=L"\Vert A-XY\Vert_F",
                    #yscale=log10,
                    xgridstyle=:dash, ygridstyle=:dash))
    lines!(rt₁, sqrt.(2*obj₁); color="#389826", linewidth=3, linestyle=:dashdot,
            label="BBPG")
    lines!(rt₂, sqrt.(2*obj₂); color="#FFC633", linewidth=2, linestyle=:solid,
            label="ALBreI")
    lines!(rt₃, sqrt.(2*obj₃); color="#CB3C33", linewidth=2, linestyle=:solid,
            label="ABLBreI")
    limits!(0, 120, 0.8, 1.0)
    axislegend("Algorithms"; merge=true)
    current_figure()
end

with_theme(speed, theme=theme_dark())
save("plot/synthesis_data.png", speed())

UndefVarError: UndefVarError: obj₀ not defined